<h3 style="color:#FDF6EC;font-size:40px;font-family:'Comic Sans MS';text-align:center;background-color : #05595B; border-
radius: 5px 5px;"><strong>Who churned ? Who are our best customers ? Who are we going to target next ?</strong></h3>

<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Content</strong></h3>

- `customer_id` : The customer unique id.

- `transaction date` : the date when the transaction was done.

- `Amount of purchase`: How many quantities each customer had.

- `Response`: the response information of each of the customers. It is a binary variable indicating whether the customer responded to a campaign or not.

* [1. Importing Libraries](#1)


* [2. Loading&Merging Data](#2)


* [3. Data Preparation and Preprocessing](#3)


* [4. RFM Metrics](#4)


* [5. RFM Score](#5)


* [6. Segmenting customers using RFM score.](#6)


* [7. Interpretation of Descriptive Statistics of Segments](#7)
 
 
* [8. Who Churned ?](#8.1)


* [9. Who are our 20 best customers ?](#8.2)


* [10. Who are we going to target next ?](#8.3)

<a id="1"></a>
<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Importing Libraries</strong></h3>

In [1]:
#Import libraries
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings # Uyarılar
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

<a id="2"></a>

<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Loading&Merging data</strong></h3>

In [2]:
transaction = pd.read_csv('Retail_Data_Transactions.csv')
response = pd.read_csv('Retail_Data_Response.csv')

df = pd.merge(transaction, response, on='customer_id', how = 'inner')

<a id="3"></a>

<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Data Preparation and Preprocessing</strong></h3>

In [3]:
# see the first five rows
df.head()

,customer_id,trans_date,tran_amount,response
0,CS5295,11-Feb-13,35,1
1,CS5295,25-Nov-14,54,1
2,CS5295,22-May-13,37,1
3,CS5295,24-Dec-12,36,1
4,CS5295,22-Mar-13,98,1


In [4]:
# see the information of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124969 entries, 0 to 124968
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  124969 non-null  object
 1   trans_date   124969 non-null  object
 2   tran_amount  124969 non-null  int64 
 3   response     124969 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 4.8+ MB


In [5]:
# converting the date to datetime type instead of object type
df['trans_date'] =  pd.to_datetime(df['trans_date'])

In [6]:
print("The data has ", df.shape[0] ,"rows and ", df.shape[1], "Columns")

The data has  124969 rows and  4 Columns


In [7]:
# see some statistical about the data
df.describe()

,tran_amount,response
count,124969.000000,124969.000000
mean,64.995143,0.110763
std,22.860059,0.313840
min,10.000000,0.000000
25%,47.000000,0.000000
50%,65.000000,0.000000
75%,83.000000,0.000000
max,105.000000,1.000000


In [8]:
# check for null values
df.isnull().sum()

customer_id    0
trans_date     0
tran_amount    0
response       0
dtype: int64

<a id="4"></a>

<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Creating RFM Metrics</strong></h3>

- **Recency** - Time since last order or engaged with product.

- **Frequency** - Total number of transactions made by customer or average time between transactions.

- **Monetary** - Total or average amount spent by the customer.

In [9]:
df.head()

,customer_id,trans_date,tran_amount,response
0,CS5295,2013-02-11,35,1
1,CS5295,2014-11-25,54,1
2,CS5295,2013-05-22,37,1
3,CS5295,2012-12-24,36,1
4,CS5295,2013-03-22,98,1


In [10]:
print("The First transaction was at ",df['trans_date'].min(),"The Last Transaction was at ", df['trans_date'].max())

The First transaction was at  2011-05-16 00:00:00 The Last Transaction was at  2015-03-16 00:00:00


In [11]:
# adding the recency, frequency and monetary
today_date = dt.datetime(2015,3,18)

rfm = df.groupby('customer_id').agg({'trans_date': lambda x : (today_date - x.max()).days,
                                     'customer_id': lambda x : x.nunique(),
                                     'tran_amount': lambda x : x.sum(),
                                      'response': lambda x : x.nunique()})

rfm.columns=['Recency', 'Frequency', 'Monetary','response' ]
rfm = rfm[rfm['Monetary'] > 0]
rfm.head()

,Recency,Frequency,Monetary,response
customer_id,,,,
CS1112,63,1,1012,1
CS1113,37,1,1490,1
CS1114,34,1,1432,1
CS1115,13,1,1659,1
CS1116,205,1,857,1


<a id="5"></a>
<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Creating RFM Score</strong></h3>

In [12]:
#Date from customer's last purchase.The nearest date gets 5 and the furthest date gets 1.
rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Total number of purchases.The least frequency gets 1 and the maximum frequency gets 5.
rfm["frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

#Total spend by the customer.The least money gets 1, the most money gets 5.
rfm["monetary_score"]= pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])

rfm.head()

,Recency,Frequency,Monetary,response,recency_score,frequency_score,monetary_score
customer_id,,,,,,,
CS1112,63,1,1012,1,3,1,2
CS1113,37,1,1490,1,4,1,4
CS1114,34,1,1432,1,4,1,4
CS1115,13,1,1659,1,5,1,5
CS1116,205,1,857,1,1,1,2


In [13]:
# If we concatenate R and F scores we will simply get RFM score  
rfm['RFM_SCORE'] = (rfm['recency_score'].astype('str') + rfm['frequency_score'].astype('str'))
rfm.head()

,Recency,Frequency,Monetary,response,recency_score,frequency_score,monetary_score,RFM_SCORE
customer_id,,,,,,,,
CS1112,63,1,1012,1,3,1,2,31
CS1113,37,1,1490,1,4,1,4,41
CS1114,34,1,1432,1,4,1,4,41
CS1115,13,1,1659,1,5,1,5,51
CS1116,205,1,857,1,1,1,2,11


<a id="6"></a>
<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Segmenting customers using RFM score.</strong></h3>

In [14]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,response,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
customer_id,,,,,,,,,
CS1112,63,1,1012,1,3,1,2,31,about_to_sleep
CS1113,37,1,1490,1,4,1,4,41,promising
CS1114,34,1,1432,1,4,1,4,41,promising
CS1115,13,1,1659,1,5,1,5,51,new_customers
CS1116,205,1,857,1,1,1,2,11,hibernating


<a id="7"></a>
<h3 style="color:#05595B;font-size:35px;font-family:newtimeroman;text-align:center;"><strong>Interpretation of Descriptive Statistics of Segments
</strong></h3>

In [15]:
rfm_stat = rfm[["segment", "Recency", "Frequency","Monetary"]].groupby('segment').agg(['mean','count','max']).round()
rfm_stat

Recency            Frequency           Monetary        \
                       mean count  max      mean count max     mean count   
segment                                                                     
about_to_sleep         57.0   589   74         1   589   1   1429.0   589   
at_Risk               153.0  1074  661         1  1074   1   1160.0  1074   
cant_loose            182.0   698  680         1   698   1    563.0   698   
champions              10.0   498   19         1   498   1    966.0   498   
hibernating           146.0   960  521         1   960   1   1332.0   960   
loyal_customers        44.0   994   74         1   994   1    898.0   994   
need_attention         57.0   271   74         1   271   1   1416.0   271   
new_customers          10.0   317   19         1   317   1   1444.0   317   
potential_loyalists    19.0  1177   41         1  1177   1   1447.0  1177   
promising              30.0   306   41         1   306   1   1458.0   306   

                           
                      max  
segment                    
about_to_sleep       2413  
at_Risk              2647  
cant_loose           1081  
champions            2439  
hibernating          2513  
loyal_customers      2326  
need_attention       2933  
new_customers        2330  
potential_loyalists  2527  
promising            2354

<a id="8.1"></a>
<h3 style="color:#05595B;font-size:44px;font-family:newtimeroman;text-align:center;"><strong>Who churned ?
</strong></h3>

**Churned Customers** who spent more than 100 days without buying anything from us.

In [49]:
churned = rfm[rfm['Recency'] > 100]
churned['Recency'].head(20)

customer_id
CS1116    205
CS1117    259
CS1123    111
CS1126    181
CS1129    108
CS1134    111
CS1135    337
CS1139    113
CS1142    163
CS1143    214
CS1144    158
CS1145    144
CS1155    128
CS1158    104
CS1167    121
CS1168    113
CS1172    260
CS1182    104
CS1193    285
CS1197    163
Name: Recency, dtype: int64

### These are the customers that already churned.

<a id="8.2"></a>
<h3 style="color:#05595B;font-size:44px;font-family:newtimeroman;text-align:center;"><strong>Who are our 20 best customers?
</strong></h3>

In [21]:
retention = rfm[rfm['Recency'] < 100]
best_customer = retention.sort_values(by=['RFM_SCORE', 'Monetary'], ascending=[False,True])
best_customer.head(20)

,Recency,Frequency,Monetary,response,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
customer_id,,,,,,,,,
CS8077,17,1,164,1,5,5,1,55,champions
CS7889,19,1,198,1,5,5,1,55,champions
CS8763,2,1,237,1,5,5,1,55,champions
CS8562,13,1,250,1,5,5,1,55,champions
CS7856,18,1,257,1,5,5,1,55,champions
CS8799,9,1,278,1,5,5,1,55,champions
CS8607,16,1,292,1,5,5,1,55,champions
CS8595,19,1,318,1,5,5,1,55,champions
CS8861,12,1,321,1,5,5,1,55,champions


<a id="8.3"></a>
<h3 style="color:#05595B;font-size:44px;font-family:newtimeroman;text-align:center;"><strong>Who are we going to target next?
</strong></h3>

### we are going to target only our retention customers who respond to our campaigns, and need to rebuy from us.

> First segment we'll target is **About_to_sleep** : Last purchase was `57 days ago` , and they are `589 customers`, and the `average spend is 1429`. The amount spent was very good. They need to be reminded of our brand to repurchase from us again.

In [48]:
target = rfm[rfm['response'] == 1]
target_customers = target[target['Recency'] < 100]
sleepy_customers = target_customers[target_customers['segment'] == 'about_to_sleep']
sleepy_customers.head()

,Recency,Frequency,Monetary,response,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
customer_id,,,,,,,,,
CS1112,63,1,1012,1,3,1,2,31,about_to_sleep
CS1121,43,1,1524,1,3,1,4,31,about_to_sleep
CS1122,44,1,1156,1,3,1,3,31,about_to_sleep
CS1137,57,1,1380,1,3,1,4,31,about_to_sleep
CS1141,55,1,1778,1,3,1,5,31,about_to_sleep


> Second segment we'll target is **need_attention** :  Last purchase was 57 days ago , and they are 271 customers, and the average spend is 1416. The amount spent was very good. They need to be reminded of our brand to repurchase from us again.

In [46]:
need_attention_customers = target_customers[target_customers['segment'] == 'need_attention']
need_attention_customers.head()

,Recency,Frequency,Monetary,response,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
customer_id,,,,,,,,,
CS3876,50,1,1993,1,3,3,5,33,need_attention
CS3887,53,1,1366,1,3,3,4,33,need_attention
CS3897,66,1,2141,1,3,3,5,33,need_attention
CS3899,51,1,1260,1,3,3,3,33,need_attention
CS3901,46,1,1145,1,3,3,3,33,need_attention
